In [1]:
import pandas as pd

try:
    df_2022 = pd.read_csv('ipl_2022_deliveries.csv')
    df_2023 = pd.read_csv('ipl_2023_deliveries.csv')
    df_2024 = pd.read_csv('ipl_2024_deliveries.csv')

    # Merge the dataframes
    merged_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=False)
    print(merged_df)

except FileNotFoundError:
    print("One or more CSV files not found. Please check the file names and paths.")
except Exception as e:
    print(f"An error occurred: {e}")

       match_id  season  match_no          date  \
0        202201    2022         1  Mar 26, 2022   
1        202201    2022         1  Mar 26, 2022   
2        202201    2022         1  Mar 26, 2022   
3        202201    2022         1  Mar 26, 2022   
4        202201    2022         1  Mar 26, 2022   
...         ...     ...       ...           ...   
17048    202474    2024        74  May 26, 2024   
17049    202474    2024        74  May 26, 2024   
17050    202474    2024        74  May 26, 2024   
17051    202474    2024        74  May 26, 2024   
17052    202474    2024        74  May 26, 2024   

                                 venue batting_team bowling_team  innings  \
0             Wankhede Stadium, Mumbai          CSK          KKR        1   
1             Wankhede Stadium, Mumbai          CSK          KKR        1   
2             Wankhede Stadium, Mumbai          CSK          KKR        1   
3             Wankhede Stadium, Mumbai          CSK          KKR        1   
4 

In [2]:

player_stats = merged_df.groupby('striker')['runs_of_bat'].agg(
    zeroes=lambda x: (x == 0).sum(),
    ones=lambda x: (x == 1).sum(),
    twos=lambda x: (x == 2).sum(),
    threes=lambda x: (x == 3).sum(),
    fours=lambda x: (x == 4).sum(),
    sixes=lambda x: (x == 6).sum(),
    total_runs=lambda x: x.sum()
).reset_index()

# Wickets taken calculation
wickets_df = merged_df[merged_df['wicket_type'].notnull()].groupby('bowler')['wicket_type'].count().reset_index(name='wickets_taken')
player_stats = pd.merge(player_stats, wickets_df, left_on='striker', right_on='bowler', how='left')
player_stats = player_stats.drop(columns='bowler').rename(columns={'batsman': 'player'})
player_stats['wickets_taken'] = player_stats['wickets_taken'].fillna(0).astype(int)

player_stats

,striker,zeroes,ones,twos,threes,fours,sixes,total_runs,wickets_taken
0,Abbott,3,1,0,0,0,1,7,1
1,Abdul Basith,0,1,0,0,0,0,1,0
2,Abdul Samad,95,95,14,0,25,22,355,0
3,Abhijeet Tomar,7,0,0,0,1,0,4,0
4,Abhinav Manohar,69,75,6,0,21,10,231,0
...,...,...,...,...,...,...,...,...,...
272,Yudhvir Singh,8,4,1,0,1,2,22,4
273,Zampa,7,10,0,0,0,0,10,9
274,de Kock,243,238,38,1,86,40,901,0
275,du Plessis,398,437,70,5,156,70,1636,0


In [3]:
try:
    df_ipl_players = pd.read_csv('ipl_2025_players.csv', sep=',')
    print(df_ipl_players)
except FileNotFoundError:
    print("File not found. Please check the file name and path.")
except pd.errors.ParserError:
    print("Error parsing the file. Please check the file format (tab-delimited).")
except Exception as e:
    print(f"An error occurred: {e}")

                                           Player Name Team
0                                      Ruturaj Gaikwad  CSK
1                                         Devon Conway  CSK
2                                      Rachin Ravindra  CSK
3                                       Rahul Tripathi  CSK
4                                          Shivam Dube  CSK
..                                                 ...  ...
106                                    Pat Cummins (C)  SRH
107                                     Mohammed Shami  SRH
108                                      Harshal Patel  SRH
109                                       Rahul Chahar  SRH
110  Credits: https://blog.league11.in/articles/ipl...  NaN

[111 rows x 2 columns]


In [4]:
try:
    df_ipl_players = df_ipl_players.drop(index=110)  # Drop row with index 109 (110th row)
    print(df_ipl_players)
except NameError:
    print("df_ipl_players not found. Please ensure the dataframe is loaded correctly.")
except IndexError:
    print("Index out of bounds.  Check if row 110 exists in the dataframe.")
except Exception as e:
    print(f"An error occurred: {e}")

         Player Name Team
0    Ruturaj Gaikwad  CSK
1       Devon Conway  CSK
2    Rachin Ravindra  CSK
3     Rahul Tripathi  CSK
4        Shivam Dube  CSK
..               ...  ...
105  Abhinav Manohar  SRH
106  Pat Cummins (C)  SRH
107   Mohammed Shami  SRH
108    Harshal Patel  SRH
109     Rahul Chahar  SRH

[110 rows x 2 columns]


In [5]:

def classify_player(player_name, merged_df):
    total_matches = len(merged_df[(merged_df['bowler'] == player_name) | (merged_df['striker'] == player_name)]['match_id'].unique())
    bowled_matches = len(merged_df[merged_df['bowler'] == player_name]['match_id'].unique())

    if total_matches == 0 :
      return "Not Played"

    ratio = bowled_matches / total_matches
    if ratio == 1:
        return 'Bowler'
    elif ratio >= 0.5:
        return 'All-rounder'
    elif ratio < 0.5:
        return 'Batsman'

player_classifications = []
for player in player_stats['striker']:
    player_classifications.append(classify_player(player, merged_df))

player_stats['Player_Type'] = player_classifications
player_stats

,striker,zeroes,ones,twos,threes,fours,sixes,total_runs,wickets_taken,Player_Type
0,Abbott,3,1,0,0,0,1,7,1,Bowler
1,Abdul Basith,0,1,0,0,0,0,1,0,Batsman
2,Abdul Samad,95,95,14,0,25,22,355,0,Batsman
3,Abhijeet Tomar,7,0,0,0,1,0,4,0,Batsman
4,Abhinav Manohar,69,75,6,0,21,10,231,0,Batsman
...,...,...,...,...,...,...,...,...,...,...
272,Yudhvir Singh,8,4,1,0,1,2,22,4,Bowler
273,Zampa,7,10,0,0,0,0,10,9,Bowler
274,de Kock,243,238,38,1,86,40,901,0,Batsman
275,du Plessis,398,437,70,5,156,70,1636,0,Batsman


In [6]:
# Rename 'striker' column to 'player'
player_stats = player_stats.rename(columns={'striker': 'player'})

# Select specific columns
player_stats = player_stats[['player','zeroes', 'ones', 'twos', 'threes', 'fours', 'sixes', 'wickets_taken', 'total_runs', 'Player_Type']]

player_stats

,player,zeroes,ones,twos,threes,fours,sixes,wickets_taken,total_runs,Player_Type
0,Abbott,3,1,0,0,0,1,1,7,Bowler
1,Abdul Basith,0,1,0,0,0,0,0,1,Batsman
2,Abdul Samad,95,95,14,0,25,22,0,355,Batsman
3,Abhijeet Tomar,7,0,0,0,1,0,0,4,Batsman
4,Abhinav Manohar,69,75,6,0,21,10,0,231,Batsman
...,...,...,...,...,...,...,...,...,...,...
272,Yudhvir Singh,8,4,1,0,1,2,4,22,Bowler
273,Zampa,7,10,0,0,0,0,9,10,Bowler
274,de Kock,243,238,38,1,86,40,0,901,Batsman
275,du Plessis,398,437,70,5,156,70,0,1636,Batsman


In [7]:
player_stats['Number of balls played'] = player_stats['zeroes'] + player_stats['ones'] + player_stats['twos'] + player_stats['threes'] + player_stats['fours'] + player_stats['sixes']

# Move the new column to the desired position
cols = list(player_stats.columns)
cols.insert(cols.index('wickets_taken'), cols.pop(cols.index('Number of balls played')))
player_stats = player_stats[cols]

player_stats

,player,zeroes,ones,twos,threes,fours,sixes,Number of balls played,wickets_taken,total_runs,Player_Type
0,Abbott,3,1,0,0,0,1,5,1,7,Bowler
1,Abdul Basith,0,1,0,0,0,0,1,0,1,Batsman
2,Abdul Samad,95,95,14,0,25,22,251,0,355,Batsman
3,Abhijeet Tomar,7,0,0,0,1,0,8,0,4,Batsman
4,Abhinav Manohar,69,75,6,0,21,10,181,0,231,Batsman
...,...,...,...,...,...,...,...,...,...,...,...
272,Yudhvir Singh,8,4,1,0,1,2,16,4,22,Bowler
273,Zampa,7,10,0,0,0,0,17,9,10,Bowler
274,de Kock,243,238,38,1,86,40,646,0,901,Batsman
275,du Plessis,398,437,70,5,156,70,1136,0,1636,Batsman


In [8]:
import random
random.seed(0)
# Calculate global distribution
global_distribution = player_stats[['zeroes','ones', 'twos', 'threes', 'fours', 'sixes']].sum()
total_balls_global = global_distribution.sum()
global_distribution = global_distribution / total_balls_global

cutoff = 60

# Function to augment player stats
def augment_stats(row):
    balls_played = row['Number of balls played']
    if balls_played < cutoff:
        remaining_balls = cutoff - balls_played
        # Adjust distribution for bowlers
        if row['Player_Type'] == 'Bowler':
            # Example adjustment: lower probabilities of scoring high runs
            adjusted_global = global_distribution.copy()
            adjusted_global[3] *= 0.2  # Reduce probability of fours
            adjusted_global[4] *= 0.1  # Reduce probability of sixes

            #Ensure probabilities sum to 1
            adjusted_global = adjusted_global / adjusted_global.sum()

            for _ in range(remaining_balls):
                outcome = random.choices(adjusted_global.index, weights=adjusted_global.values, k=1)[0]
                row[outcome] += 1

        else:
            for _ in range(remaining_balls):
                outcome = random.choices(global_distribution.index, weights=global_distribution.values, k=1)[0]
                row[outcome] += 1
    return row

# Apply augmentation
player_stats = player_stats.apply(augment_stats, axis=1)

player_stats

C:\Users\abhij\AppData\Local\Temp\ipykernel_2576\745032248.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adjusted_global[3] *= 0.2  # Reduce probability of fours
C:\Users\abhij\AppData\Local\Temp\ipykernel_2576\745032248.py:19: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  adjusted_global[3] *= 0.2  # Reduce probability of fours
C:\Users\abhij\AppData\Local\Temp\ipykernel_2576\745032248.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use

,player,zeroes,ones,twos,threes,fours,sixes,Number of balls played,wickets_taken,total_runs,Player_Type
0,Abbott,22,26,7,1,0,4,5,1,7,Bowler
1,Abdul Basith,15,26,4,0,12,3,1,0,1,Batsman
2,Abdul Samad,95,95,14,0,25,22,251,0,355,Batsman
3,Abhijeet Tomar,31,18,1,0,5,5,8,0,4,Batsman
4,Abhinav Manohar,69,75,6,0,21,10,181,0,231,Batsman
...,...,...,...,...,...,...,...,...,...,...,...
272,Yudhvir Singh,30,22,1,0,3,4,16,4,22,Bowler
273,Zampa,28,25,2,0,1,4,17,9,10,Bowler
274,de Kock,243,238,38,1,86,40,646,0,901,Batsman
275,du Plessis,398,437,70,5,156,70,1136,0,1636,Batsman


In [9]:
player_stats['Number of runs'] = (player_stats['ones'] + 2 * player_stats['twos'] + 3 * player_stats['threes'] + 4 * player_stats['fours'] + 6 * player_stats['sixes'])
player_stats['Number of balls played'] = (player_stats['zeroes'] + player_stats['ones'] +  player_stats['twos'] + player_stats['threes'] + player_stats['fours'] + player_stats['sixes'])

# Move the new column to the desired position
cols = list(player_stats.columns)
cols.insert(cols.index('wickets_taken'), cols.pop(cols.index('Number of runs')))
player_stats = player_stats[cols]
player_stats = player_stats.drop(columns=['total_runs'])
player_stats

,player,zeroes,ones,twos,threes,fours,sixes,Number of balls played,Number of runs,wickets_taken,Player_Type
0,Abbott,22,26,7,1,0,4,60,67,1,Bowler
1,Abdul Basith,15,26,4,0,12,3,60,100,0,Batsman
2,Abdul Samad,95,95,14,0,25,22,251,355,0,Batsman
3,Abhijeet Tomar,31,18,1,0,5,5,60,70,0,Batsman
4,Abhinav Manohar,69,75,6,0,21,10,181,231,0,Batsman
...,...,...,...,...,...,...,...,...,...,...,...
272,Yudhvir Singh,30,22,1,0,3,4,60,60,4,Bowler
273,Zampa,28,25,2,0,1,4,60,57,9,Bowler
274,de Kock,243,238,38,1,86,40,646,901,0,Batsman
275,du Plessis,398,437,70,5,156,70,1136,1636,0,Batsman


In [10]:
# Create a mapping of player names from player_stats to df_ipl_players
name_mapping = {}
for player_name in player_stats['player']:
    for ipl_name in df_ipl_players['Player Name']:
        if player_name in ipl_name:
          name_mapping[ipl_name] = player_name
          break

# Update player names in df_ipl_players based on the mapping
df_ipl_players['Player'] = df_ipl_players['Player Name'].replace(name_mapping)
df_ipl_players

,Player Name,Team,Player
0,Ruturaj Gaikwad,CSK,Gaikwad
1,Devon Conway,CSK,Conway
2,Rachin Ravindra,CSK,Ravindra
3,Rahul Tripathi,CSK,Tripathi
4,Shivam Dube,CSK,Shivam Dube
...,...,...,...
105,Abhinav Manohar,SRH,Abhinav Manohar
106,Pat Cummins (C),SRH,Cummins
107,Mohammed Shami,SRH,Shami
108,Harshal Patel,SRH,Harshal Patel


In [11]:
def arrange_batting_order(team_players):
    team_stats = player_stats[player_stats['player'].isin(team_players)]

    # Sort players based on player type and stats
    batters = team_stats[team_stats['Player_Type'] == 'Batsman'].sort_values(by='Number of runs', ascending=False)
    allrounders = team_stats[team_stats['Player_Type'] == 'All-rounder'].sort_values(by=['Number of runs', 'wickets_taken'], ascending=False)
    bowlers = team_stats[team_stats['Player_Type'] == 'Bowler'].sort_values(by='wickets_taken', ascending=False)

    # Concatenate the sorted groups
    arranged_team = pd.concat([batters, allrounders, bowlers])

    return arranged_team['player'].tolist()

# Example usage (assuming df_ipl_players contains 'Team Name' and 'Player Name')
teams = df_ipl_players['Team'].unique()
for team in teams:
  team_players = df_ipl_players[df_ipl_players['Team'] == team]['Player'].tolist()
  batting_order = arrange_batting_order(team_players)
  print(f"Batting order for {team}: {batting_order}")

Batting order for CSK: ['Gaikwad', 'Shivam Dube', 'Conway', 'Tripathi', 'Dhoni', 'Ravindra', 'Khaleel Ahmed', 'Ashwin', 'Jadeja', 'Noor Ahmad', 'Mukesh Choudhary']
Batting order for DC: ['Rahul', 'Tristan Stubbs', 'Abishek Porel', 'Fraser-McGurk', 'Harry Brook', 'Ashutosh Sharma', 'Axar', 'Mohit Sharma', 'Kuldeep Yadav', 'Mukesh Kumar', 'Starc']
Batting order for GT: ['Shubman Gill', 'Buttler', 'Sai Sudharsan', 'Rahul Tewatia', 'Shahrukh Khan', 'Rutherford', 'Washington Sundar', 'Rashid Khan', 'Siraj', 'Rabada', 'Prasidh']
Batting order for KKR: ['Venkatesh Iyer', 'de Kock', 'Rinku Singh', 'Rahane', 'Ramandeep Singh', 'Russell', 'Narine', 'Harshit Rana', 'Nortje', 'Markande', 'Vaibhav Arora']
Batting order for LSG: ['Pooran', 'Miller', 'Markram', 'Pant', 'Mayank', 'Arshin Kulkarni', 'Shahbaz Ahmed', 'Mitchell Marsh', 'Avesh Khan', 'Ravi Bishnoi', 'Mohsin Khan']
Batting order for MI: ['Suryakumar Yadav', 'Tilak Varma', 'Rohit', 'Naman Dhir', 'Hardik Pandya', 'Will Jacks', 'Boult', 'Bumr

In [12]:
team_batting_orders = {}
for team in teams:
  team_players = df_ipl_players[df_ipl_players['Team'] == team]['Player'].tolist()
  batting_order = arrange_batting_order(team_players)
  team_batting_orders[team] = batting_order

# Accessing batting orders:
# Example:
print(team_batting_orders['KKR'])

['Venkatesh Iyer', 'de Kock', 'Rinku Singh', 'Rahane', 'Ramandeep Singh', 'Russell', 'Narine', 'Harshit Rana', 'Nortje', 'Markande', 'Vaibhav Arora']


In [13]:
def simulate_match(team1, team2, team_batting_orders, player_stats):
    import random
    random.seed(0)
    # Toss and decide batting order
    toss_winner = random.choice([team1, team2])
    batting_first = toss_winner
    bowling_first = team2 if toss_winner == team1 else team1

    # Initialize scorecard with initial values
    scorecard = {
        batting_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {
                team_batting_orders[batting_first][0]: {'runs': 0, 'balls': 0, 'out': False},  # Batsman 1
                team_batting_orders[batting_first][1]: {'runs': 0, 'balls': 0, 'out': False}  # Batsman 2
            },
            'yet_to_bat': team_batting_orders[batting_first][2:],
            'current_batsmen': [team_batting_orders[batting_first][0], team_batting_orders[batting_first][1]],
            'bowler_stats': {}
        },
        bowling_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {},
            'yet_to_bat': [],
            'current_batsmen': [],
            'bowler_stats': {}
        }
    }

    def simulate_ball(batsman, bowler):
        batsman_stats = player_stats[player_stats['player'] == batsman].iloc[0]
        bowler_stats = player_stats[player_stats['player'] == bowler].iloc[0]

        total_balls_faced = batsman_stats['Number of balls played']
        outcomes = [0, 1, 2, 3, 4, 6, 'wicket']
        probabilities = [
            batsman_stats['zeroes'] / total_balls_faced,
            batsman_stats['ones'] / total_balls_faced,
            batsman_stats['twos'] / total_balls_faced,
            batsman_stats['threes'] / total_balls_faced,
            batsman_stats['fours'] / total_balls_faced,
            batsman_stats['sixes'] / total_balls_faced,
            bowler_stats['wickets_taken'] / total_balls_faced  # Probability of a wicket
        ]

        # Adjust probabilities to add randomness
        probabilities = [p + random.uniform(-0.1, 0.1) for p in probabilities]
        probabilities = [max(0, p) for p in probabilities]  # Ensure probabilities are non-negative
        probabilities = [p / sum(probabilities) for p in probabilities]  # Normalize probabilities

        # Randomly select an outcome based on probabilities
        outcome = random.choices(outcomes, weights=probabilities)[0]
        return outcome

    def simulate_innings(batting_team, bowling_team, batting_order, target=None):

        # Initialize current_batsmen for the batting team at the start of each innings
        scorecard[batting_team]['current_batsmen'] = [batting_order[0], batting_order[1]]  # Initialize with the first two batsmen
        scorecard[batting_team]['yet_to_bat'] = batting_order[2:]  # Update yet_to_bat
        
        # Initialize player_scores for all batsmen in the order
        for batsman in batting_order:
            scorecard[batting_team]['player_scores'].setdefault(batsman, {'runs': 0, 'balls': 0, 'out': False}) # Initialize before usage

        while scorecard[batting_team]['wickets'] < 10 and scorecard[batting_team]['overs'] < 20:
            bowler = team_batting_orders[bowling_team][-1 - (scorecard[batting_team]['overs'] // 4) % 5]
            scorecard[batting_team]['bowler_stats'].setdefault(bowler, {'overs': 0, 'balls': 0, 'wickets': 0, 'runs_conceded': 0})

            for ball in range(6):
                scorecard[batting_team]['balls'] += 1
                if scorecard[batting_team]['balls'] > 120:
                    break

                batsman = scorecard[batting_team]['current_batsmen'][0]
                non_striker = scorecard[batting_team]['current_batsmen'][1]

                outcome = simulate_ball(batsman, bowler)

                if outcome == 'wicket':
                    scorecard[batting_team]['wickets'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['out'] = True

                    scorecard[batting_team]['bowler_stats'][bowler]['wickets'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1

                    if scorecard[batting_team]['wickets'] < 10:
                        next_batsman = scorecard[batting_team]['yet_to_bat'].pop(0) if scorecard[batting_team]['yet_to_bat'] else None
                        if next_batsman:
                            scorecard[batting_team]['player_scores'][next_batsman] = {'runs': 0, 'balls': 0, 'out': False}
                            scorecard[batting_team]['current_batsmen'][0] = next_batsman
                        else:
                            break  # All players are out before wickets are all 10
                    else:
                        break
                else:
                    scorecard[batting_team]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1

                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['runs_conceded'] += outcome

                    # Check if target is reached (for 2nd innings only) - PLACE IT HERE
                    if target is not None and scorecard[batting_team]['runs'] >= target:
                        return  # Stop the innings if target is reached

                    if outcome in (1, 3, 5):  # Rotate strike on odd runs
                        scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]

                if (ball + 1) % 6 == 0:  # Rotate strike at the end of over
                    scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]
                    scorecard[batting_team]['overs'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['overs'] = scorecard[batting_team]['bowler_stats'][bowler]['balls'] // 6


    def print_scorecard(team):
        print(f"\n{team}: {scorecard[team]['runs']}/{scorecard[team]['wickets']} ({scorecard[team]['overs']}.{scorecard[team]['balls'] % 6} overs)")
        print("Player Scores:")
        for player, stats in scorecard[team]['player_scores'].items():
          if stats['balls'] > 0:  # Only show players who have batted 
            not_out_symbol = "*" if not stats['out'] else ""
            print(f"  {player}: {stats['runs']}{not_out_symbol} ({stats['balls']})")
       
        if scorecard[team]['yet_to_bat']:
            print("\nYet to bat:")
            print(", ".join(scorecard[team]['yet_to_bat']))
        
        # Bowling statistics
        print("\nBowling Statistics:")
        for bowler, stats in scorecard[team if team == bowling_first else batting_first]['bowler_stats'].items():  # Access the bowling team's stats
            print(f"  {bowler}: {stats['overs']}.{stats['balls'] % 6} overs, {stats['wickets']} wickets, {stats['runs_conceded']} runs")

        
        


    print(f"{batting_first} is batting first")
    simulate_innings(batting_first, bowling_first, team_batting_orders[batting_first])
    target = scorecard[batting_first]['runs'] + 1
    simulate_innings(bowling_first, batting_first, team_batting_orders[bowling_first], target=target)

    if scorecard[bowling_first]['runs'] >= target:
        winner = bowling_first
        result = f"{winner} won by {10 - scorecard[bowling_first]['wickets']} wickets"
    elif scorecard[bowling_first]['runs'] == target - 1:
        winner = None
        result = "Match tied"
    else:
        winner = batting_first
        result = f"{winner} won by {target - 1 - scorecard[bowling_first]['runs']} runs"

    print_scorecard(batting_first)
    print_scorecard(bowling_first)
    print(f"\n{result}")

    return scorecard, result
    # Example simulation between KKR and RCB
scorecard, result = simulate_match("KKR", "RCB", team_batting_orders, player_stats)

RCB is batting first

RCB: 168/4 (20.0 overs)
Player Scores:
  Kohli: 46 (28)
  Rajat Patidar: 78 (45)
  Padikkal: 13 (17)
  Tim David: 2 (4)
  Philip Salt: 19* (10)
  Livingstone: 10* (16)

Yet to bat:
Krunal Pandya, Bhuvneshwar, Yash Dayal, Hazlewood, Rasikh Salam

Bowling Statistics:
  Vaibhav Arora: 4.0 overs, 0 wickets, 38 runs
  Markande: 4.0 overs, 0 wickets, 31 runs
  Nortje: 4.0 overs, 1 wickets, 55 runs
  Harshit Rana: 4.0 overs, 3 wickets, 17 runs
  Narine: 4.0 overs, 0 wickets, 27 runs

KKR: 119/10 (13.0 overs)
Player Scores:
  Venkatesh Iyer: 19 (9)
  de Kock: 0 (1)
  Rinku Singh: 41 (30)
  Rahane: 10 (8)
  Ramandeep Singh: 0 (1)
  Russell: 23 (8)
  Narine: 11 (15)
  Harshit Rana: 0 (1)
  Nortje: 0 (1)
  Markande: 6 (6)
  Vaibhav Arora: 9* (4)

Bowling Statistics:
  Rasikh Salam: 4.0 overs, 2 wickets, 46 runs
  Hazlewood: 4.0 overs, 2 wickets, 23 runs
  Yash Dayal: 4.0 overs, 4 wickets, 39 runs
  Bhuvneshwar: 1.0 overs, 2 wickets, 11 runs

RCB won by 49 runs


In [14]:
try:
    df_matches = pd.read_csv('ipl_2024_matches.csv')
    print(df_matches.head()) # Display first few rows of the dataframe
except FileNotFoundError:
    print("File 'ipl_2024_matches.csv' not found. Please upload the file.")
except pd.errors.ParserError:
    print("Error parsing the CSV file. Please check the file format.")
except Exception as e:
    print(f"An error occurred: {e}")

   match_id  season        date  match_no  \
0    202401    2024  2024-03-22         1   
1    202402    2024  2024-03-23         2   
2    202403    2024  2024-03-23         3   
3    202404    2024  2024-03-24         4   
4    202405    2024  2024-03-24         5   

                                               venue       city team1 team2  \
0           MA Chidambaram Stadium, Chepauk, Chennai    Chennai   RCB   CSK   
1  Maharaja Yadavindra Singh International Cricke...     Mohali    DC  PBKS   
2                              Eden Gardens, Kolkata    Kolkata   KKR   SRH   
3                     Sawai Mansingh Stadium, Jaipur     Jaipur    RR   LSG   
4                   Narendra Modi Stadium, Ahmedabad  Ahmedabad    GT    MI   

  toss_winner toss_decision  ...  innings2_wickets  winning_team  margin  \
0         RCB           bat  ...               4.0           CSK     6.0   
1        PBKS         field  ...               6.0          PBKS     4.0   
2         SRH         fiel

In [15]:
# Keep only specified columns
df_matches = df_matches[['match_no', 'team1', 'team2']]
print(df_matches)

    match_no team1 team2
0          1   RCB   CSK
1          2    DC  PBKS
2          3   KKR   SRH
3          4    RR   LSG
4          5    GT    MI
..       ...   ...   ...
69        70    RR   KKR
70        71   SRH   KKR
71        72   RCB    RR
72        73   SRH    RR
73        74   SRH   KKR

[74 rows x 3 columns]


In [16]:
import random
random.seed(0)
def simulate_match_summary(team1, team2, team_batting_orders, player_stats):
    # Toss and decide batting order
    toss_winner = random.choice([team1, team2])
    batting_first = toss_winner
    bowling_first = team2 if toss_winner == team1 else team1

    # Initialize scorecard with initial values
    scorecard = {
        batting_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {
                team_batting_orders[batting_first][0]: {'runs': 0, 'balls': 0, 'out': False},  # Batsman 1
                team_batting_orders[batting_first][1]: {'runs': 0, 'balls': 0, 'out': False}  # Batsman 2
            },
            'yet_to_bat': team_batting_orders[batting_first][2:],
            'current_batsmen': [team_batting_orders[batting_first][0], team_batting_orders[batting_first][1]],
            'bowler_stats': {}
        },
        bowling_first: {
            'runs': 0,
            'wickets': 0,
            'overs': 0,
            'balls': 0,
            'player_scores': {},
            'yet_to_bat': [],
            'current_batsmen': [],
            'bowler_stats': {}
        }
    }

    def simulate_ball(batsman, bowler):
        batsman_stats = player_stats[player_stats['player'] == batsman].iloc[0]
        bowler_stats = player_stats[player_stats['player'] == bowler].iloc[0]

        total_balls_faced = batsman_stats['Number of balls played']
        outcomes = [0, 1, 2, 3, 4, 6, 'wicket']
        probabilities = [
            batsman_stats['zeroes'] / total_balls_faced,
            batsman_stats['ones'] / total_balls_faced,
            batsman_stats['twos'] / total_balls_faced,
            batsman_stats['threes'] / total_balls_faced,
            batsman_stats['fours'] / total_balls_faced,
            batsman_stats['sixes'] / total_balls_faced,
            bowler_stats['wickets_taken'] / total_balls_faced  # Probability of a wicket
        ]

        # Adjust probabilities to add randomness
        probabilities = [p + random.uniform(-0.1, 0.1) for p in probabilities]
        probabilities = [max(0, p) for p in probabilities]  # Ensure probabilities are non-negative
        probabilities = [p / sum(probabilities) for p in probabilities]  # Normalize probabilities

        # Randomly select an outcome based on probabilities
        outcome = random.choices(outcomes, weights=probabilities)[0]
        return outcome

    def simulate_innings(batting_team, bowling_team, batting_order, target=None):

        # Initialize current_batsmen for the batting team at the start of each innings
        scorecard[batting_team]['current_batsmen'] = [batting_order[0], batting_order[1]]  # Initialize with the first two batsmen
        scorecard[batting_team]['yet_to_bat'] = batting_order[2:]  # Update yet_to_bat

        # Initialize player_scores for all batsmen in the order
        for batsman in batting_order:
            scorecard[batting_team]['player_scores'].setdefault(batsman, {'runs': 0, 'balls': 0, 'out': False}) # Initialize before usage

        while scorecard[batting_team]['wickets'] < 10 and scorecard[batting_team]['overs'] < 20:
            bowler = team_batting_orders[bowling_team][-1 - (scorecard[batting_team]['overs'] // 4) % 5]
            scorecard[batting_team]['bowler_stats'].setdefault(bowler, {'overs': 0, 'balls': 0, 'wickets': 0, 'runs_conceded': 0})

            for ball in range(6):
                scorecard[batting_team]['balls'] += 1
                if scorecard[batting_team]['balls'] > 120:
                    break

                batsman = scorecard[batting_team]['current_batsmen'][0]
                non_striker = scorecard[batting_team]['current_batsmen'][1]

                outcome = simulate_ball(batsman, bowler)

                if outcome == 'wicket':
                    scorecard[batting_team]['wickets'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1
                    scorecard[batting_team]['player_scores'][batsman]['out'] = True
                    scorecard[batting_team]['bowler_stats'][bowler]['wickets'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1

                    if scorecard[batting_team]['wickets'] < 10:
                        next_batsman = scorecard[batting_team]['yet_to_bat'].pop(0) if scorecard[batting_team]['yet_to_bat'] else None
                        if next_batsman:
                            scorecard[batting_team]['player_scores'][next_batsman] = {'runs': 0, 'balls': 0, 'out': False}
                            scorecard[batting_team]['current_batsmen'][0] = next_batsman
                        else:
                            break  # All players are out before wickets are all 10
                    else:
                        break  
                else:
                    scorecard[batting_team]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['runs'] += outcome
                    scorecard[batting_team]['player_scores'][batsman]['balls'] += 1

                    scorecard[batting_team]['bowler_stats'][bowler]['balls'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['runs_conceded'] += outcome

                    # Check if target is reached (for 2nd innings only) - PLACE IT HERE
                    if target is not None and scorecard[batting_team]['runs'] >= target:
                        return  # Stop the innings if target is reached

                    if outcome in (1, 3, 5):  # Rotate strike on odd runs
                        scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]

                if (ball + 1) % 6 == 0:  # Rotate strike at the end of over
                    scorecard[batting_team]['current_batsmen'] = scorecard[batting_team]['current_batsmen'][::-1]
                    scorecard[batting_team]['overs'] += 1
                    scorecard[batting_team]['bowler_stats'][bowler]['overs'] = scorecard[batting_team]['bowler_stats'][bowler]['balls'] // 6


    def print_scorecard(team):
        print(f"\n{team}: {scorecard[team]['runs']}/{scorecard[team]['wickets']} ({scorecard[team]['overs']}.{scorecard[team]['balls'] % 6} overs)")
    simulate_innings(batting_first, bowling_first, team_batting_orders[batting_first])
    target = scorecard[batting_first]['runs'] + 1
    simulate_innings(bowling_first, batting_first, team_batting_orders[bowling_first], target=target)

    if scorecard[bowling_first]['runs'] >= target:
        winner = bowling_first
        result = f"{winner} won by {10 - scorecard[bowling_first]['wickets']} wickets"
    elif scorecard[bowling_first]['runs'] == target - 1:
        winner = None
        result = "Match tied"
    else:
        winner = batting_first
        result = f"{winner} won by {target - 1 - scorecard[bowling_first]['runs']} runs"

    print_scorecard(batting_first)
    print_scorecard(bowling_first)
    print(f"\n{result}")

    if "won by" in result:
        winning_team = result.split("won by")[0].strip()  # Extract the winner's name
    elif "Match tied" in result:
        winning_team = None  # No winner in case of a tie
    else:
        winning_team = result.split("won by")[0].strip()
    
    return winning_team, scorecard, result

def simulate_season(df_matches, team_batting_orders, player_stats):
    points_table = pd.DataFrame(index=df_ipl_players['Team'].unique(), columns=['Points', 'NRR', 'Total Runs Scored', 'Total Overs Played', 'Total Runs Conceded', 'Total Overs Bowled'])
    points_table.fillna(0, inplace=True)

    for i in range(min(70, len(df_matches))):
        match = df_matches.iloc[i]
        team1 = match['team1']
        team2 = match['team2']

        print(f"\nMATCH {i+1}: {team1} VS {team2}")
        winning_team, scorecard, result = simulate_match_summary(team1, team2, team_batting_orders, player_stats)
        # Award Points
        if "won by" in result:
            winner = result.split("won by")[0].strip()
            points_table.loc[winner, 'Points'] += 2
        elif "Match tied" in result:
            points_table.loc[team1, 'Points'] += 1
            points_table.loc[team2, 'Points'] += 1

          # Update total runs and overs
        points_table.loc[team1, 'Total Runs Scored'] += scorecard[team1]['runs']
        points_table.loc[team1, 'Total Overs Played'] += scorecard[team1]['overs'] + (scorecard[team1]['balls'] % 6) / 10
        points_table.loc[team1, 'Total Runs Conceded'] += scorecard[team2]['runs']
        points_table.loc[team1, 'Total Overs Bowled'] += scorecard[team2]['overs'] + (scorecard[team2]['balls'] % 6) / 10

        points_table.loc[team2, 'Total Runs Scored'] += scorecard[team2]['runs']
        points_table.loc[team2, 'Total Overs Played'] += scorecard[team2]['overs'] + (scorecard[team2]['balls'] % 6) / 10
        points_table.loc[team2, 'Total Runs Conceded'] += scorecard[team1]['runs']
        points_table.loc[team2, 'Total Overs Bowled'] += scorecard[team1]['overs'] + (scorecard[team1]['balls'] % 6) / 10

    # Calculate final NRR for each team after all matches
    points_table['NRR'] = (points_table['Total Runs Scored'] / points_table['Total Overs Played']) - (points_table['Total Runs Conceded'] / points_table['Total Overs Bowled'])
    points_table['NRR'] = points_table['NRR'].fillna(0) # Replace NaN with 0

    # Correct Overs Format
    points_table['Total Overs Played'] = points_table['Total Overs Played'].astype(int) + (points_table['Total Overs Played'] % 1 * 10 / 6).round(1) % 1 * 6 /10
    points_table['Total Overs Bowled'] = points_table['Total Overs Bowled'].astype(int) + (points_table['Total Overs Bowled'] % 1 * 10 / 6).round(1) % 1 * 6 /10

    # Display only Points and NRR columns

    points_table = points_table[['Points', 'NRR']]
    # Sort points table
    points_table = points_table.sort_values(by=['Points', 'NRR'], ascending=[False, False])
    print(points_table)
    return points_table

points_table = simulate_season(df_matches, team_batting_orders, player_stats)

def simulate_playoffs(points_table):
    # Get top 4 teams
    top_4_teams = points_table.head(4).index.tolist()

    playoff_schedule = pd.DataFrame(columns=['Match', 'Team 1', 'Team 2', 'Winner'])

    # Qualifier 1
    team1_qualifier1 = top_4_teams[0]
    team2_qualifier1 = top_4_teams[1]
    winner_qualifier1, scorecard, result = simulate_match_summary(team1_qualifier1, team2_qualifier1, team_batting_orders, player_stats)
    
    # Handle tie in Qualifier 1
    if result == "Match tied":
        winner_qualifier1 = team1_qualifier1  # Higher position in points table wins

    # Eliminator
    team1_eliminator = top_4_teams[2]
    team2_eliminator = top_4_teams[3]
    winner_eliminator, scorecard, result = simulate_match_summary(team1_eliminator, team2_eliminator, team_batting_orders, player_stats)
    
    # Handle tie in Eliminator
    if result == "Match tied":
        winner_eliminator = team1_eliminator  # Higher position in points table wins

    # Qualifier 2
    loser_qualifier1 = team2_qualifier1 if winner_qualifier1 == team1_qualifier1 else team1_qualifier1
    winner_qualifier2, scorecard, result = simulate_match_summary(winner_eliminator, loser_qualifier1, team_batting_orders, player_stats)

    # Handle tie in Qualifier 2
    if result == "Match tied":
        winner_qualifier2 = loser_qualifier1  # Higher position wins between teams who previously had a higher position in pointstable wins

    # Final
    ipl_champion, scorecard, result = simulate_match_summary(winner_qualifier1, winner_qualifier2, team_batting_orders, player_stats)
    if result == "Match tied":
        ipl_champion = winner_qualifier2 if winner_qualifier2 == team1_qualifier1 else winner_qualifier1  # Winner of Qualifier 1 wins in case of tie in the final


    playoff_schedule = pd.DataFrame({
        'Match': ['Qualifier 1', 'Eliminator', 'Qualifier 2', 'Final'],
        'Team 1': [team1_qualifier1, team1_eliminator, winner_eliminator, winner_qualifier1],
        'Team 2': [team2_qualifier1, team2_eliminator, loser_qualifier1, winner_qualifier2],
        'Winner': [winner_qualifier1, winner_eliminator, winner_qualifier2, ipl_champion]
    })

    print(playoff_schedule)
    print(f"\nThe IPL Champions is : {ipl_champion}")

# Assuming 'points_table' is available from the previous code block
simulate_playoffs(points_table)



MATCH 1: RCB VS CSK


C:\Users\abhij\AppData\Local\Temp\ipykernel_2576\2211581251.py:153: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  points_table.fillna(0, inplace=True)



CSK: 173/4 (20.0 overs)

RCB: 103/10 (12.3 overs)

CSK won by 70 runs

MATCH 2: DC VS PBKS

PBKS: 162/10 (17.0 overs)

DC: 123/10 (16.1 overs)

PBKS won by 39 runs

MATCH 3: KKR VS SRH


C:\Users\abhij\AppData\Local\Temp\ipykernel_2576\2211581251.py:172: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  points_table.loc[team1, 'Total Overs Played'] += scorecard[team1]['overs'] + (scorecard[team1]['balls'] % 6) / 10
C:\Users\abhij\AppData\Local\Temp\ipykernel_2576\2211581251.py:179: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.3' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  points_table.loc[team2, 'Total Overs Bowled'] += scorecard[team1]['overs'] + (scorecard[team1]['balls'] % 6) / 10



SRH: 145/10 (15.0 overs)

KKR: 146/4 (19.4 overs)

KKR won by 6 wickets

MATCH 4: RR VS LSG

RR: 145/10 (16.2 overs)

LSG: 146/1 (17.3 overs)

LSG won by 9 wickets

MATCH 5: GT VS MI

MI: 182/4 (20.0 overs)

GT: 130/8 (20.0 overs)

MI won by 52 runs

MATCH 6: PBKS VS RCB

RCB: 116/10 (17.4 overs)

PBKS: 119/5 (14.2 overs)

PBKS won by 5 wickets

MATCH 7: CSK VS GT

CSK: 181/6 (20.0 overs)

GT: 105/10 (13.5 overs)

CSK won by 76 runs

MATCH 8: SRH VS MI

SRH: 128/10 (14.5 overs)

MI: 131/3 (17.3 overs)

MI won by 7 wickets

MATCH 9: RR VS DC

DC: 121/10 (15.4 overs)

RR: 123/2 (13.0 overs)

RR won by 8 wickets

MATCH 10: RCB VS KKR

RCB: 126/10 (16.5 overs)

KKR: 130/6 (16.0 overs)

KKR won by 4 wickets

MATCH 11: LSG VS PBKS

PBKS: 94/10 (10.4 overs)

LSG: 95/1 (11.2 overs)

LSG won by 9 wickets

MATCH 12: SRH VS GT

SRH: 119/10 (14.0 overs)

GT: 82/10 (14.2 overs)

SRH won by 37 runs

MATCH 13: DC VS CSK

DC: 140/10 (19.5 overs)

CSK: 141/4 (18.2 overs)

CSK won by 6 wickets

MATCH 1

In [17]:
def calculate_player_stats(scorecard):
    player_runs = {}
    player_wickets = {}

    for team, team_data in scorecard.items():
        for player, stats in team_data['player_scores'].items():
            player_runs[player] = player_runs.get(player, 0) + stats['runs']
        for bowler, stats in team_data['bowler_stats'].items():
            player_wickets[bowler] = player_wickets.get(bowler, 0) + stats['wickets']

    return player_runs, player_wickets

# Example usage within the simulate_season or simulate_playoffs functions:
player_runs, player_wickets = calculate_player_stats(scorecard) # Assuming scorecard is the combined stats from the season

#Find the orange cap holder
orange_cap_holder = max(player_runs, key=player_runs.get)

#Find the purple cap holder
purple_cap_holder = max(player_wickets, key=player_wickets.get)

print(f"Orange cap holder is: {orange_cap_holder}")
print(f"Purple cap holder is: {purple_cap_holder}")

Orange cap holder is: Rajat Patidar
Purple cap holder is: Yash Dayal
